In [ ]:
using Geant4
using G4Visualization
using Geant4.SystemOfUnits
#---Choose the Makie backend
#using GLMakie
using WGLMakie
#using CairoMakie
using GeometryBasics

In [ ]:
include(joinpath(@__DIR__, "../../examples/basic/B2/DetectorB2a.jl"))

In [ ]:
#---Define Simulation Data struct------------------------------------------------------------------
mutable struct B2aSimData <: G4JLSimulationData
  tracks::Vector{Vector{Point3{Float64}}}   # vector of vector of step positions
end

In [ ]:
#---Step action------------------------------------------------------------------------------------
function stepaction(step::G4Step, app::G4JLApplication)::Nothing
  tracks = app.simdata.tracks
  p = GetPosition(GetPostStepPoint(step))
  push!(tracks[end], Point3{Float64}(x(p),y(p),z(p)))
  return
end
#---Tracking pre-action---------------------------------------------------------------------------- 
function pretrackaction(track::G4Track, app::G4JLApplication)::Nothing
  tracks = app.simdata.tracks
  p = GetPosition(track)[]
  push!(tracks, [Point3{Float64}(x(p),y(p),z(p))])
  return
end
#---begin-event-action---------------------------------------------------------------------------- 
function beginevent(::G4Event, app::G4JLApplication)::Nothing
  empty!(app.simdata.tracks)
  return
end

In [ ]:
#---Particle Gun initialization--------------------------------------------------------------------
function gun_initialize(gen::G4JLParticleGun, det::G4JLDetector)
  pg = GetGun(gen)
  SetParticleByName(pg, "proton")
  SetParticleEnergy(pg, 3GeV)
  SetParticleMomentumDirection(pg, G4ThreeVector(0,0,1))
  SetParticlePosition(pg, G4ThreeVector(0, 0, -det.worldZHalfLength))
end
Geant4.getInitializer(::G4JLParticleGun) = gun_initialize

In [ ]:
#---Create the Application-------------------------------------------------------------------------
app = G4JLApplication(detector=B2aDetector(nChambers=5),          # detector with parameters
                      simdata = B2aSimData([]),                   # simulation data structure
                      runmanager_type = G4RunManager,             # what RunManager to instantiate
                      physics_type = FTFP_BERT,                   # what physics list to instantiate
                      generator_type = G4JLParticleGun,           # what primary generator to instantiate
                      stepaction_method = stepaction,             # step action method
                      begineventaction_method = beginevent,       # begin-event action (initialize per-event data)
                      pretrackaction_method = pretrackaction)     # pre-tracking action
              
#---Configure, Initialize and Run------------------------------------------------------------------                      
configure(app)
initialize(app)
beamOn(app, 8)

In [ ]:
world = GetWorldVolume()
tracks = app.simdata.tracks;

In [ ]:
beamOn(app, 1)

fig = Figure(resolution=(1024,1024))
s = LScene(fig[1,1])
draw!(s, world)
for t in tracks
    lines!(t)
end
display(fig)